In [150]:
using JuMP, Gurobi
import Random
import Plots

Generate Data

In [151]:
seed_value = 39
function generate_data(seed_value, num_requests)
    Random.seed!(seed_value)

    # num_requests & num_nodes
    num_nodes = 2 * num_requests + 2 # Includes two depot nodes

    # service times
    service_times = rand(1:3, 2*num_requests) 
    service_times = vcat(0, service_times, 0) # add depot service time 

    # locations
    locations = [rand(0:10, 2) for _ in 1:(2*num_requests + 1)] 
    locations = vcat(locations, [locations[1]]) # add end depot 

    # loads
    loads = [1 for _ in 1:num_requests] # pickup
    loads = vcat(loads, -loads) # add delivery
    loads = vcat(0, loads, 0) # add depot

    # depot
    depot_start = 1

    # generate_distance_matrix
    function generate_distance_matrix(locations)
        num_nodes = length(locations)
        
        travel_times = zeros(num_nodes, num_nodes)
        
        for i in 1:num_nodes
            for j in 1:num_nodes
                if i != j
                    dx = locations[i][1] - locations[j][1]
                    dy = locations[i][2] - locations[j][2]
                    travel_times[i, j] = round(sqrt(dx^2 + dy^2))
                end
            end
        end
        
        return travel_times
    end

    # travel_times
    travel_times = generate_distance_matrix(locations)
    
    # time windows
    time_windows = []
    pick_up = []
    delivery = []
    c = 10 * num_requests
    for i in 1:num_requests
        A_p = rand(0:c)
        A_d = A_p + service_times[i + num_requests + 1] + travel_times[i+1, i + num_requests + 1]
        push!(pick_up, (A_p, A_p + c/2))
        push!(delivery, (A_d, A_d + c/2))
    end
    time_windows = vcat((0, 0), pick_up, delivery, (0, 20*num_requests)) # add depot 

    return time_windows, travel_times, service_times, loads
end

generate_data (generic function with 1 method)

Functions for DP

In [152]:
# find visited nodes
function find_indices(set, a, num_nodes)
    indices = Int[]
    mask = bitstring(set)
    for (index, char) in enumerate(mask)
        if char == '1'
            push!(indices, length(mask) - index + 1)
        end
    end

    one_indices = reverse(indices)
    
    if a == 1
        return one_indices
    else
        zero_indices = [i for i in 1:num_nodes]
        zero_indices = setdiff(zero_indices, one_indices)
        return zero_indices
    end
end

# calculate current load
function cur_load(set, loads)
    indices = find_indices(set, 1, length(loads))
    total_load = 0
    for index in indices
        total_load += loads[index]
    end

    return total_load
end
    
# count number of nodes visited
function count_ones(set) 
    count = 0
    mask = bitstring(set)
    mask = string(mask)

    for char in mask
        if char == '1'
            count += 1
        end
    end
    return count
end

# partial ordering
function partial_order(vector)
    indices = Int[]
    for i in 1:length(vector)
        count = 0
        for j in 1:length(vector)
            if vector[j][1] <= vector[i][1] && vector[j][2] <= vector[i][2] && i != j && vector[j][1:2] != vector[i][1:2]
                count += 1
            end
        end
        if count == 0 
            push!(indices, i)
        end
    end
    new = [vector[index] for index in indices]
    new = unique(new)
    return new
end

# find trajectory
function find_trajectory(dict, preceding_label_num, num_nodes)
    last_label = preceding_label_num
    traj = []
    
    for i in 1:num_nodes
        for (key, value) in dict
            if value[1] == last_label
                push!(traj, (key[2],(key[3][1], key[3][2])))
                last_label = value[2] # update label 
            end
        end
    end
    push!(traj, ((1, (0, 0))))
    traj = reverse(traj)
    return traj
end

# criteria 1

# criteria 2
function criteria_2(set, node, num_requests)
    if num_requests + 2 <= node <= 2*num_requests + 1 # delivery node
        if (set & (1 << (node - num_requests - 1))) != 0 # node visited
            return true
        else
            return false
        end
    elseif 2 <= node <= num_requests + 1 # pickup node
        return true
    else # depot
        if set == ((1 << (2*num_requests + 1) - 1))
            return true
        else
            return false
        end
    end
end

# criteria 3
function criteria_3(set, node, num_requests, loads, capacity)
    if 2 <= node <= num_requests + 1
        if cur_load(set, loads) + loads[node] <= capacity
            return true
        else
            return false
        end
    else
        return true
    end
end

# criteria 4 
function criteria_4(set, node, time_windows, travel_times, service_times)
    # Initialize minimum values
    min_value = Inf
    min_index = 0
    new_time_windows = [(time_windows[unvisited], unvisited) for unvisited in find_indices(set, 0, length(service_times)) if unvisited != node]
    # Iterate through the vector
    for time_window in new_time_windows
        second_element = time_window[1][2]
        if second_element < min_value
            min_value = second_element
            min_index = time_window[2]
        end
    end
    if min_index == 0
        return true
    else 
        if time_windows[node][1] + service_times[node] + travel_times[node, min_index] <= min_value
            return true
        else
            return false
        end
    end
end

# find t_cur_node, 이제 label의 list를 반환 
function t_node_test(dict, set, node, label)
    filtered_values = filter(kv -> kv[1][1:3] == (set, node, label), dict)

    if isempty(filtered_values)
        return 
    else
        time = getindex(first(filtered_values)[1][3], 1)
        return time
    end
end

# criteria 5
function criteria_5(dict, set, cur_node, next_node, label, time_windows, travel_times, service_times)
    if t_node_test(dict, set, cur_node, label) == nothing
        return false
    elseif t_node_test(dict, set, cur_node, label) + service_times[cur_node] + travel_times[cur_node, next_node] <= time_windows[next_node][2]
        return true
    else
        return false
    end
end

# criteria 6
function criteria_6(dict, set, cur_node, next_node, label, time_windows, travel_times, service_times)
    # Initialize minimum values
    min_value = Inf
    min_index = 0
    num_nodes = length(service_times)
    unvisited_indices = find_indices(set, 0, num_nodes)
    new_time_windows = [(time_windows[unvisited], unvisited) for unvisited in unvisited_indices if unvisited != next_node] # unvisited and not next_node
    # Iterate through the vector
    for time_window in new_time_windows
        second_element = time_window[1][2]
        if second_element < min_value
            min_value = second_element
            min_index = time_window[2]
        end
    end
    if min_index == 0
        return true
    else
        if t_node_test(dict, set, cur_node, label) == nothing
            return false
        else 
            t_next_node = max(time_windows[next_node][1], t_node_test(dict, set, cur_node, label) + service_times[cur_node] + travel_times[cur_node, next_node])
            if t_next_node + service_times[next_node] + travel_times[next_node, min_index] <= time_windows[min_index][2]
                return true
            else
                return false
            end
        end
    end
end

# criteria 7
function criteria_7(dict, set, cur_node, next_node, label, time_windows, travel_times, service_times, num_requests)
    min_value = Inf
    min_index = 0
    second_min_value = Inf
    second_min_index = 0
    unvisited_indices = find_indices(set, 0, length(service_times))
    new_time_windows = [(time_windows[unvisited], unvisited) for unvisited in unvisited_indices if unvisited ≠ next_node && num_requests + 2 <= unvisited <= 2*num_requests + 1] # unvisited pickup and not next_node
    for time_window in new_time_windows
        second_element = time_window[1][2]

        if second_element < min_value
            second_min_value = min_value
            second_min_index = min_index

            min_value = second_element
            min_index = time_window[2]
        elseif second_element < second_min_value
            second_min_value = second_element
            second_min_index = time_window[2]
        end
    end
    if min_index == 0 || second_min_index == 0 
        return true
    else
        if t_node_test(dict, set, cur_node, label) == nothing
            return false
        else
            t_next_node = max(time_windows[next_node][1], t_node_test(dict, set, cur_node, label) + service_times[cur_node] + travel_times[cur_node, next_node])
            if max(time_windows[min_index][1], t_next_node + service_times[next_node] + travel_times[next_node, min_index]) + service_times[min_index] + travel_times[min_index, second_min_index] <= time_windows[second_min_index][2] || max(time_windows[second_min_index][1], t_next_node + service_times[next_node] + travel_times[next_node, second_min_index]) + service_times[second_min_index] + travel_times[second_min_index, min_index] <= time_windows[min_index][2]
                return true
            else
                return false
            end
        end
    end
end

# criteria 8
function criteria_8(dict, set, cur_node, next_node, label, time_windows, travel_times, service_times, num_requests)
    min_value = Inf
    min_index = 0
    second_min_value = Inf
    second_min_index = 0
    num_nodes = length(service_times)
    unvisited_indices = find_indices(set, 0, num_nodes)
    new_time_windows = [(time_windows[unvisited], unvisited) for unvisited in unvisited_indices if unvisited ≠ next_node && 2 <= unvisited <= num_requests + 1 && criteria_2(set, unvisited, num_requests)] # unvisited delivery which its origin visited and not next_node
    for time_window in new_time_windows
        second_element = time_window[1][2]

        if second_element < min_value
            second_min_value = min_value
            second_min_index = min_index

            min_value = second_element
            min_index = time_window[2]
        elseif second_element < second_min_value
            second_min_value = second_element
            second_min_index = time_window[2]
        end
    end
    if min_index == 0 || second_min_index == 0 
        return true
    else
        if t_node_test(dict, set, cur_node, label) == nothing
            return false
        else 
            t_next_node = max(time_windows[next_node][1], t_node_test(dict, set, cur_node, label) + service_times[cur_node] + travel_times[cur_node, next_node])
            if max(time_windows[min_index][1], t_next_node + service_times[next_node] + travel_times[next_node, min_index]) + service_times[min_index] + travel_times[min_index, second_min_index] <= time_windows[second_min_index][2] || max(time_windows[second_min_index][1], t_next_node + service_times[next_node] + travel_times[next_node, second_min_index]) + service_times[second_min_index] + travel_times[second_min_index, min_index] <= time_windows[min_index][2]
                return true
            else
                return false
            end
        end
    end
end

criteria_8 (generic function with 1 method)

PDPTW forward DP

In [153]:
# state: (set, current position, label(time, cost)), value: (label number, preceding label number)
function pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)
    memo = Dict{Tuple{Int, Int, Tuple{Float64, Float64}}, Tuple{Int, Int}}() # (set, current position, label(time, cost)), (label number, preceding label number)
    num_nodes = length(service_times)
    num_requests = div(num_nodes - 2, 2)
    label_num = 1
    memo[1, 1, (0, 0)] = (1, 0)
    for k in 1:num_nodes
        if k == 1 # first iteration
            for j in 2:num_requests+1
                if loads[1] + loads[j] <= capacity
                    label_num += 1
                    memo[1 | 1 << (j - 1), j, (max(time_windows[j][1], time_windows[1][1] + travel_times[1, j]), travel_times[1, j])] = (label_num, 0)
                end
            end
        elseif k == num_nodes # last iteration 
            ans = Inf
            last_label = 0
            final_labels = filter(kv -> kv[1][1:2] == ((1 << num_nodes) - 1, num_nodes), memo)
            for (key, value) in final_labels
                if ans > key[3][2]
                    ans = key[3][2]
                    last_label = value[2]
                end
            end
            trajectory = find_trajectory(memo, last_label, num_nodes)
            for (key, value) in final_labels
                if value[2] == last_label
                    push!(trajectory, (key[2], key[3]))
                end
            end
            return trajectory, ans
        else # 2 <= k <= num_nodes - 1 iteration 
            for mask in 1:(1 << (num_nodes - 1)) - 1
                if count_ones(mask) == k && string(bitstring(mask))[end] == '1' # for sets of size k
                    for j in 2:num_nodes
                        if (mask & (1 << (j - 1))) == 0  # if node j is not visited, criteria 1
                            if criteria_2(mask, j, num_requests) && criteria_3(mask, j, num_requests, loads, capacity) && criteria_4(mask, j, time_windows, travel_times, service_times)
                                for i in 2:num_nodes
                                    if (mask & (1 << (i - 1))) != 0 # if node i is visited
                                        filtered_values = filter(kv -> kv[1][1:2] == (mask, i), memo)
                                        candidates = Vector{Any}()
                                        for (key, value) in filtered_values
                                            label = key[3]
                                            if criteria_5(memo, mask, i, j, label, time_windows, travel_times, service_times) && criteria_6(memo, mask, i, j, label, time_windows, travel_times, service_times) && criteria_7(memo, mask, i, j, label, time_windows, travel_times, service_times, num_requests) && criteria_8(memo, mask, i, j, label, time_windows, travel_times, service_times, num_requests)
                                                preceding_label_num = memo[mask, i, (key[3][1], key[3][2])][1]
                                                push!(candidates, (max(time_windows[j][1], key[3][1] + service_times[i] + travel_times[i, j]), key[3][2] + travel_times[i, j], preceding_label_num)) # (time, cost, label number, preceding label number)
                                            end
                                        end
                                        if isempty(candidates)
                                            nothing
                                        else
                                            candidates = unique(candidates)
                                            filtered_labels = partial_order(candidates)
                                            for label in filtered_labels
                                                label_num += 1
                                                memo[mask | 1 << (j - 1), j, (label[1], label[2])] = (label_num, label[3])
                                            end
                                        end
                                    end
                                end
                            end
                        end
                    end
                end
            end
        end
    end
end

pdptw_dynamic_programming_forward (generic function with 1 method)

PDPTW IP

In [154]:
function build_pdptw_model(time_windows, travel_times, service_times, capacity)
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    # num_requests & num_nodes
    num_nodes = size(time_windows, 1)
    num_requests = div(num_nodes - 2, 2)

    depot_start = 1
    depot_end = num_nodes
    # loads
    loads = [1 for _ in 1:num_requests] # pickup
    loads = vcat(loads, -loads) # add delivery
    loads = vcat(0, loads, 0) # add depot


    # pickup & delivery points 
    P = [i for i in 2:num_requests + 1]
    D = [i for i in num_requests + 2:2*num_requests + 1]

    # variables
    @variable(model, T[1:num_nodes] >= 0) # start time variable
    @variable(model, x[1:num_nodes, 1:num_nodes], Bin) # decision variable
    @variable(model, Q[1:num_nodes] >=0) # load variable 

    # objective function: minimize total travel time 
    @objective(model, Min, sum(travel_times .* x)) 
    
    # constraints
    # (6.18), (6.19)
    for i in P # for all i in P
        @constraint(model, sum(x[i, j] for j = 1:num_nodes) == 1)
        @constraint(model, (sum(x[i, j] for j = 1:num_nodes) - sum(x[num_requests+i, j] for j = 1:num_nodes)) == 0) 
    end
    
    # (6.20), (6.21), (6.22)
    @constraint(model, sum(x[1, j] for j in 1:num_nodes) == 1) # start depot
    for i in vcat(P, D)
        @constraint(model, (sum(x[i, j] for j = 1:num_nodes) - sum(x[j, i] for j = 1:num_nodes)) == 0)
    end
    @constraint(model, sum(x[j, num_nodes] for j in 1:num_nodes) == 1) # end depot
    
    # time window constraint (6.23)
    for i in 1:num_nodes
        for j in 1:num_nodes
            @constraint(model, T[j] >= (T[i] + service_times[i] + travel_times[i, j]) * x[i, j])
        end
    end
    
    # (6.27)
    for i in 1:num_nodes
        @constraint(model, time_windows[i][1] <= T[i])
        @constraint(model, time_windows[i][2] >= T[i])
    end

    # precedence constraint (6.25)
    for i in P
        @constraint(model, T[num_requests+i] - T[i] - travel_times[i, num_requests+i] - service_times[i] >= 0)
    end

    # load constraint 
    # (6.24)
    for i in 1:num_nodes
        for j in 1:num_nodes
            @constraint(model, Q[j] >= (loads[j] + Q[i]) * x[i,j]) 
        end
    end
    
    # (6.28)
    for i in 1:num_nodes
        @constraint(model, Q[i] <= min(capacity, capacity + loads[i]))
        @constraint(model, max(0, loads[i]) <= Q[i])
    end

    #return model
    # Solve the optimization problem
    optimize!(model)

    # Check if the optimization was successful and a solution was found
    return model
end

build_pdptw_model (generic function with 1 method)

Gurobi == DP

In [164]:
function test_dp(num_requests)
    capacity = 3
    feasible_seed = []
    same = []
    dif = []
    seed_value = 1
    feasible_count = 0
    while feasible_count < 1
        seed_value += 1
        time_windows, travel_times, service_times, loads = generate_data(seed_value, num_requests)
        model = build_pdptw_model(time_windows, travel_times, service_times, capacity)
        if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
            feasible_count += 1
            push!(feasible_seed, seed_value)
            if objective_value(model) == pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)[2]
                push!(same, (objective_value(model), pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)[2]))
            else
                # compare route 
                traj = route(model, num_requests)
                push!(dif, (objective_value(model), pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)[2], seed_value, (traj, pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)[1])))
            end
        end
    end
    println(same)
    println(dif)
    println(length(dif))
end
test_dp(10)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-26
Any[(76.0, 76.0)]
Any[]
0


In [156]:
function fix(seed_value, num_requests)
    capacity =10
    time_windows, travel_times, service_times, loads = generate_data(seed_value, num_requests)
    model = build_pdptw_model(time_windows, travel_times, service_times, capacity)
    gurobi_route = route(model, num_requests)
    dp_route_label = pdptw_dynamic_programming_forward(time_windows, travel_times, service_times, loads, capacity)[1]
    dp_route = []
    for node in dp_route_label
        push!(dp_route, node[1])
    end
    gurobi_data = []
    dp_data = []
    for i in 1:2*num_requests + 1
        push!(gurobi_data, (gurobi_route[i], time_windows[gurobi_route[i]], service_times[gurobi_route[i]], travel_times[gurobi_route[i], gurobi_route[i + 1]]))
        push!(dp_data, (dp_route[i], time_windows[dp_route[i]], service_times[dp_route[i]], travel_times[dp_route[i], dp_route[i + 1]]))
    end
    println("gurobi_data", gurobi_data)
    println("dp_data", dp_data)
end

fix (generic function with 1 method)

In [157]:
function route(model, num_requests)
    ans = [1]
    num_nodes = 2*num_requests + 2
    depot_end = num_nodes
    x = model[ :x]
    T = model[ :T]
    Q = model[ :Q]
    current_node = 1
    while current_node != depot_end
        for i in 1:num_nodes
            for j in 1:num_nodes
                if value(x[i,j]) > 0.9
                    if i == current_node
                        push!(ans, j)
                        current_node = j
                    end
                end
            end
        end
    end
    return ans
end
    

route (generic function with 1 method)